<a href="https://colab.research.google.com/github/Theoph-ay/RAG/blob/main/ChomaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain_community langchain-chroma langchain-groq langchain_classic langchain-text-splitters
!pip install -q chromadb tiktoken
!pip install -q pypdf sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB

In [2]:
!wget -q http://www.dropbox.com/s/vs6ocyvpzzncvwh/new_article.zip

In [3]:
!unzip -q new_article.zip -d new_article

In [4]:
#using this code because I alreadyt have an API key set in the colab environment
#This can be set under secrets (key button)
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')

if GROQ_API_KEY is not None:
  print("Groq api key gotten successfully")
else:
  print("Groq api key not gotten")

Groq api key gotten successfully


In [5]:
#import necessary libraries
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
loader = DirectoryLoader("/content/new_article", glob="./*.txt", loader_cls = TextLoader)
documents = loader.load()
print(f"Loaded {len(documents)} documents.")

Loaded 21 documents.


In [7]:
documents[1].page_content

"Microsoft doubles down on AI with new Bing features The company's betting the farm on generative AI\n\nMicrosoft is embarking on the next phase of Bing’s expansion. And — no surprise — it heavily revolves around AI.\n\nAt a preview event this week in New York City, Microsoft execs including Yusuf Mehdi, the CVP and consumer chief marketing officer, gave members of the press, including this reporter, a look at the range of features heading to Bing over the next few days, weeks and months.\n\nThey don’t so much reinvent the wheel as they build on what Microsoft has injected into the Bing experience over the past three months or so. Since launching Bing Chat, its AI-powered chatbot powered by OpenAI’s GPT-4 and DALL-E 2 models, Microsoft says that visitors to Bing — which has grown to exceed 100 million daily active users — have engaged in over half a billion chats and created more than 200 million images.\n\nLooking ahead, Bing will become more visual, thanks to more image- and graphic-

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [9]:
texts[0].page_content

'Databricks today announced that it has acquired Okera, a data governance platform with a focus on AI. The two companies did not disclose the purchase price. According to Crunchbase, Okera previously raised just under $30 million. Investors include Felicis, Bessemer Venture Partners, Cyber Mentor Fund, ClearSky and Emergent Ventures.'

# Creating Embedding

In [12]:
persist_directory = "db"
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory,
)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [14]:
vectordb = None

In [15]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [16]:
vectordb

In [17]:
retriever = vectordb.as_retriever()

In [19]:
docs = vectordb.similarity_search("How much money did microsoft raise?")

In [24]:
docs[0].page_content

'April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”'

In [25]:
len(docs)

4

#Make a chain

In [37]:
llm = ChatGroq(
    model= "openai/gpt-oss-120b",
    api_key = GROQ_API_KEY,
    temperature = 0.5
)

In [29]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

This chain can also be done with lcel by using | to seperate them

Check: https://colab.research.google.com/drive/1LMHJ5BzQKKP_2-awGpijhwHWRJeEaOZM

or




In [30]:
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant that helps people find information.
Answer the following question based only on the provided context.
Think step-by-step and provide a detailed answer. If you don't know the answer,
just say that you don't have enough information from the provided documents.
DO NOT use any of your own knowledge.

<context>
{context}
</context>

Question: {input}
"""

)

In [38]:
#Document chain

document_chain = create_stuff_documents_chain(llm, prompt)
#Retrieval chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [42]:
#cite sources
def process_llm_responses(llm_response):
  print(llm_response["answer"])
  print("\n\nSources:")
  for source in llm_response["context"]:
    print(source.metadata["source"])

In [44]:
#Example
query = "How much  did microsoft raise"
llm_response = retrieval_chain.invoke({"input": query})
process_llm_responses(llm_response)

**Step‑by‑step reasoning**

1. The provided documents discuss a large investment that Microsoft made earlier in the year, separate from the later venture‑capital funding.  
2. The text states: *“The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.”*  
3. No other figures for a Microsoft‑related raise or investment appear in the context.

**Answer**

According to the information given, Microsoft’s investment (i.e., the amount it put into the company) was **about $10 billion**.


Sources:
/content/new_article/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
/content/new_article/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
/content/new_article/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt
/content/new_article/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt
